### 读入tools 生成的possible matchings

In [1]:
# matchings_path = "./data/employee.json"
# answers_path = "employee_ans.json"
matchings_path = "author.json"
answers_path = "author_ans.json"
# matchings_path = "purchase.json"
# answers_path = "purchase_ans.json"
dataset_name = "author"

In [2]:
from fact import FactSet
import numpy as np
import json

with open(matchings_path, "r") as f:
    content = json.load(f)

### 初始化 chatgpt 的 tokenzier，用于估计correspondence 的cost func

In [3]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

### process 统计，factset 需要的数据

In [4]:
def process(content):
    import numpy as np
    c_set = content["correspondence_set"]
    # chatgpt 的tokens 的num list
    len_list = [len(encoding.encode(i[0][1]+i[1][1])) for i in c_set]
    matchings = content["matchings"]
    Views = []
    for match in matchings:
        view = []
        for c in c_set:
            if c in match:
                view.append(1)
            else:
                view.append(0)
        Views.append(view)
    return np.array(Views), np.array(content["prob_all"]), np.array(len_list)

### 处理后生成 facts 

In [5]:
facts, prob, len_list = process(content)
ex_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
random_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
brute_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)

## selector to select correspondence

### 初始化 selector

In [6]:
from query import QuerySelector, BaseQuerySelector, GreedyQuerySelector,RandomQuerySelector
 # 对应fact1, 3是0.8, 0.
query_selector = GreedyQuerySelector()
# selection_idxes, sub_facts, h = query_selector.select(ex_fact, 2, accuracy, cost_func=2)
random_selector = RandomQuerySelector()
base_selector = BaseQuerySelector()

## LLM答案dict

In [7]:
import json
with open(answers_path, "r") as f:
    ans_list = json.load(f)

### 参数设置

In [8]:
## params
# correspondence num
c_len = ex_fact.num_fact()

#LLM 预测准确率
p_w = 0.9
# 计算p(A_T) 和 P(A_T|v) 需要的LLM array:acc 
acc = np.array([[p_w for i in range(c_len)]])

# 每个token 一个
budget = 300 # 单位为 price of one token  
turns = 20
# k-selection 的参数
k = 4
##########################################

In [9]:
# b_turns = 2
# brute_h_list = [brute_fact.compute_entropy()]
# while b_turns>0:
#     brute_selection_idxes, _, _ = base_selector.select(brute_fact, k, acc)
#     ans_brute = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in brute_selection_idxes]
#     p_a_b,p_a_v_b = ex_fact.compute_ans_p(ans_brute, brute_selection_idxes, acc)
#     sum_p = []
#     for idx,i in enumerate(brute_fact.get_prior_p()):
#         p_post_b = i*p_a_v_b[idx]/p_a_b
#         sum_p.append(p_post_b)
#     brute_fact.set_prior_p(np.array(sum_p))
#     ex_fact.set_prior_p(np.array(sum_p))
#     random_fact.set_prior_p(np.array(sum_p))
#     brute_h_list.append(brute_fact.compute_entropy())
#     b_turns-=1

In [10]:
import numpy as np

approx_h_list=[ex_fact.compute_entropy()]


cost_sum = 0
while turns>0:
    selection_idxes, sub_facts, h = query_selector.select(ex_fact, k, acc, cost_func=1)
    # for ix in selection_idxes:
    #     cost_sum += ex_fact.len_list()[ix]
    #     ans=[1 if ans_list[ix]=="yes" else 0]
    #     p_a, p_a_v = ex_fact.compute_ans_p(ans, [ix], acc)
    #     sum_p = []
    #     for idx,i in enumerate(ex_fact.get_prior_p()):
    #         p_post = i*p_a_v[idx]/p_a
    #         sum_p.append(p_post)
    #     ex_fact.set_prior_p(np.array(sum_p))
    ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = ex_fact.compute_ans_p(ans, selection_idxes, acc)
    sum_p = []
    for idx,i in enumerate(ex_fact.get_prior_p()):
        p_post = i*p_a_v[idx]/p_a
        sum_p.append(p_post)
    ex_fact.set_prior_p(np.array(sum_p))
    approx_h_list.append(ex_fact.compute_entropy())
    turns -=1


In [11]:
all_h_list = []
cost_list = []
p_prior = random_fact.get_prior_p()
for _ in range(100):
    cost_sum_r = 0
    random_fact.set_prior_p(p_prior)
    turns_r=20
    random_h_list=[random_fact.compute_entropy()]
    random_selection_idxes, _, _  =  random_selector.select(random_fact, k, acc)
    while turns_r>0:
        # for ix_r in random_selection_idxes:
        #     cost_sum_r += ex_fact.len_list()[ix]
        #     ans_r = [1 if ans_list[ix_r]=="yes" else 0]
        #     p_a_r, p_a_v_r = random_fact.compute_ans_p(ans_r, [ix_r], acc)
        #     sum_p_r = []
        #     for idx,i in enumerate(random_fact.get_prior_p()):
        #         p_post_r = i*p_a_v_r[idx]/p_a_r
        #         sum_p_r.append(p_post_r)
        #     random_fact.set_prior_p(np.array(sum_p_r))
        ans_r = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in random_selection_idxes]
        p_a_r,p_a_v_r = random_fact.compute_ans_p(ans_r, random_selection_idxes, acc)
    
        sum_p_r = []
        for idx,i in enumerate(random_fact.get_prior_p()):
            p_post_r = i*p_a_v_r[idx]/p_a_r
            sum_p_r.append(p_post_r)
        random_fact.set_prior_p(np.array(sum_p_r))
        random_h_list.append(random_fact.compute_entropy())
        turns_r -=1
    all_h_list.append(random_h_list)
    cost_list.append(cost_sum_r)

In [12]:
approx_h_list

[2.0179569806623436,
 0.6690536564277565,
 0.12436480210466297,
 0.019241796784894613,
 0.0027203962486460308,
 0.00036375588156893146,
 4.7475444288694216e-05,
 6.062951740463603e-06,
 7.612620892531361e-07,
 9.431892306677457e-08,
 1.1561474991512041e-08,
 1.4047856462938792e-09,
 1.69440139271705e-10,
 2.0310254887040417e-11,
 2.421473354359302e-12,
 2.873695262432069e-13,
 3.3928154516766186e-14,
 3.897243409737681e-15,
 4.581531515828801e-16,
 5.369769529326141e-17,
 6.276609428343524e-18]

In [13]:
cost_sum

0

In [14]:
import numpy as np
n = np.array(all_h_list)
random_h_l = n.mean(axis=0, keepdims=True)
random_h_l

array([[2.01795698, 0.77266288, 0.51744036, 0.47412442, 0.46755725,
        0.46663131, 0.4665066 , 0.46649032, 0.46648824, 0.46648798,
        0.46648794, 0.46648794, 0.46648794, 0.46648794, 0.46648794,
        0.46648794, 0.46648794, 0.46648794, 0.46648794, 0.46648794,
        0.46648794]])

In [15]:
import json
dataset = "output"
def save_h_file(approx_name_f,random_f, approx_ob, random_ob):
    with open(f"./{dataset}/"+approx_name_f, "w") as f:
        json.dump(approx_ob, f, indent=2, ensure_ascii=False)
    with open(f"./{dataset}/"+random_f, "w") as w:
        json.dump(random_ob, w, indent=2, ensure_ascii=False)

In [16]:
name = "turns_20_k={}_{}"
save_h_file(name.format(k,f"{dataset_name}_approx.json"), name.format(k,f"{dataset_name}_random.json"),approx_h_list, random_h_l.tolist()[0])